In [1]:
import wandb
from wandb.fastai import WandbCallback

In [3]:
wandb.init(project='droughtwatch4')
wandb.init()

wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [5]:
from fastai.vision import *
from fastai import *
import tifffile

In [6]:
import torch
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Any


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-7be5be79.pth',
}

class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1) 
        x = self.classifier(x)
        return 
        
    def increase_channels(self, m, num_channels=None, copy_weights=0):


        """
        takes as input a Conv2d layer and returns the a Conv2d layer with `num_channels` input channels
        and all the previous weights copied into the new layer.
        """
        new_in_channels = 0
        # number of input channels the new module should have
        if(num_channels is not None):
            new_in_channels = num_channels
        else: 
            new_in_channels = m.in_channels + 1
        
        bias = False if m.bias is None else True
        
        # Creating new Conv2d layer
        new_m = nn.Conv2d(in_channels=new_in_channels, 
                          out_channels=m.out_channels, 
                          kernel_size=m.kernel_size, 
                          stride=m.stride, 
                          padding=m.padding,
                          bias=bias)
        
        # Copying the weights from the old to the new layer
        
        with torch.no_grad():
            new_m.weight[:, :m.in_channels, :, :] = m.weight.clone()
        
        #Copying the weights of the `copy_weights` channel of the old layer to the extra channels of the new layer
        for i in range(new_in_channels - m.in_channels):
            channel = m.in_channels + i
            with torch.no_grad():
                new_m.weight[:, channel:channel+1, :, :] = m.weight[:, copy_weights:copy_weights+1, : :].clone()
        new_m.weight = nn.Parameter(new_m.weight)

        return new_m
    
def alexnet(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> AlexNet:
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [7]:
class ImageMultiList(ImageList):
    def open(self, fn):
        "Open image in `fn`, subclass and overwrite for custom behavior."
        img=tifffile.imread(str(fn))
        img=Image(img)
        return img

In [8]:
from pathlib import Path
cur=Path.cwd()
Images=cur/'11_band_data'
class MultiDataBunch(ImageDataBunch):
    @classmethod
    def from_folder(cls, path:PathOrStr, train:PathOrStr='train', valid:PathOrStr='val',
                    valid_pct=None, seed:int=None, classes:Collection=None, **kwargs:Any)->'ImageDataBunch':
        "Create from imagenet style dataset in `path` with `train`,`valid`,`test` subfolders (or provide `valid_pct`)."
        path=Path(path)
        il = ImageMultiList.from_folder(path)
        if valid_pct is None: src = il.split_by_folder(train=train, valid=valid)
        else: src = il.split_by_rand_pct(valid_pct, seed)
        src = src.label_from_folder(classes=classes)
        return cls.create_from_ll(src, **kwargs)

In [9]:
data=MultiDataBunch.from_folder(Images, bs=256)

In [11]:
#alexnet=get_arch(101,11)
model=alexnet(True)
#model=Learner(data,arch,metrics=[accuracy], callback_fns=WandbCallback)
#model = alexnet(pretrained=True)
#model=cnn_learner(data,models.resnet34,metrics=[accuracy])

In [13]:
%%wandb
# model.unfreeze()
#model.fit_one_cycle(3)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
model.freeze_to(-3)

In [22]:
model.fit_one_cycle(5, slice(1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.160285,1.066701,0.604565,05:35
1,0.979754,1.121983,0.605864,05:35
2,0.883817,0.868039,0.664316,05:37


/home/ec2-user/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Better model found at epoch 0 with valid_loss value: 1.066701054573059.
Better model found at epoch 2 with valid_loss value: 0.8680385947227478.
Loaded best saved model from /home/ec2-user/droughtwatch/wandb/run-20210801_050501-2xh9nkil/files/bestmodel.pth


KeyboardInterrupt: 

In [ ]:
model.unfreeze()

In [ ]:
model.fit_one_cycle(15, slice(3e-3))

Okay, so a resnet101 overfits. Clearly. Not doing that anymore. We're done with this one

<hr>

In [38]:
wandb.init(project='droughtwatch', name='resnet50-run')

epoch,4
train_loss,0.51256
_runtime,5672
_timestamp,1626917084
_step,7


epoch,▁▃▅▁▃▅▆█
train_loss,█▅▂█▆▅▃▁
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


In [11]:
resnet50_11_channel=get_arch(50,11)
arch=resnet50_11_channel(True)
model=Learner(data,arch,metrics=[accuracy], callback_fns=WandbCallback)
#for some reason it used resnet18

In [12]:
model.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.823866,0.987229,0.641028,01:20
1,0.687245,0.707259,0.725181,01:20
2,0.545408,0.602339,0.766005,01:21


Better model found at epoch 0 with valid_loss value: 0.9872291088104248.
Better model found at epoch 1 with valid_loss value: 0.7072592377662659.
Better model found at epoch 2 with valid_loss value: 0.6023386120796204.
Loaded best saved model from /home/ec2-user/droughtwatch/wandb/run-20210728_073427-2hpt65hf/files/bestmodel.pth


In [13]:
results = model.validate()

In [14]:
results[1].item()

0.7660048007965088

In [15]:
wandb.log({'val_acc': results[1].item(), 'epoch':3})